In [1]:
import numpy as np
import matplotlib.pyplot as plt
#from medpy import io
import six
import pandas as pd
import SimpleITK as sitk
import seaborn as sns
from sklearn.metrics import roc_auc_score
import math

import torch
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import pytorch_lightning as pl

from torch.utils.data import DataLoader

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
"https://docs.lightly.ai/self-supervised-learning/tutorials/package/tutorial_simclr_clothing.html"
#import logging
#import os
#from typing import Any, Optional, Sequence, Tuple, Type, Union
from lightly.loss import NTXentLoss
from lightly.models.modules.heads import SimCLRProjectionHead
#import pandas as pd
import pytorch_lightning as pl
import torch
#import torch.nn.functional as F
from monai.networks.nets import EfficientNetBN, ResNet, resnet18, resnet10
from pytorch_lightning import Callback, LightningModule
from torch import nn, Tensor
from torch.optim import AdamW, Optimizer
from torch.optim.lr_scheduler import CosineAnnealingLR

#from torch.utils.tensorboard import SummaryWriter
from sklearn.linear_model import LogisticRegression
from typing import List, Any
import torchmetrics


max_epochs=200
#tb = SummaryWriter()

resnet = resnet10(pretrained=False,spatial_dims=3,n_input_channels=1,num_classes=1)#.to(device)
backbone = nn.Sequential(*list(resnet.children())[:-1])
projection_head = SimCLRProjectionHead(512, 512, 128)

class SimCLRModel(pl.LightningModule):
    def __init__(self, encoder, projection):
        super().__init__()
        self.embeddings = []
        self.labels = []
        self.classifier_acc_train=torchmetrics.Accuracy(task="binary")

        # create a ResNet backbone and remove the classification head
        
        self.backbone=encoder
        self.projection_head=projection
        #hidden_dim = resnet.fc.in_features
        self.online=nn.Sequential(nn.Linear(128,1),nn.Sigmoid())

        self.criterion = NTXentLoss()
        

    def forward(self, x):
        h = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(h)
        return z

    def training_step(self, batch, batch_idx):
        
        x0, x1,y = batch
        x0 = x0[:,None,:,:,:]
        x1 = x1[:,None,:,:,:]
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        #self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=False)
        #self.log={"train_loss": loss}
        #metrics = {"train_loss": loss}
        #self.log_dict(metrics)
        self.log_dict({"train_loss": loss},on_step=False,on_epoch=True,prog_bar=True)
        self.embeddings.append(z1)
        self.labels.append(y)
        return {"loss": loss}#, "embeddings": z1, "labels":y}
                #"log": logs}

    def on_train_epoch_end(self):
        """
        Train and log classifier
        """
        if self.current_epoch != -1:
            embeddings = self.embeddings[0]#torch.stack(self.embeddings).detach().cpu()
            labels = self.labels[0]#torch.stack(self.labels).cpu()
            
            for i in range(1,len(self.embeddings)):
                labels = torch.cat((labels, self.labels[i]), dim=0)
                embeddings = torch.cat((embeddings, self.embeddings[i]), dim=0)
            embeddings = embeddings.detach().cpu()
            labels = labels.cpu()
            #print(len(embeddings))
            #print(embeddings[0].shape)
            #embeddings, labels = self.stack_outputs(train_step_outputs)
            
            self.estimator = LogisticRegression(class_weight='balanced', max_iter=1000).fit(embeddings, labels)
            #preds, probs = self.predict_live_estimator(self.embeddings)
            
            preds = torch.from_numpy(self.estimator.predict(embeddings))
            labels=labels
            accuracy=self.classifier_acc_train(preds, labels)
            #print(accuracy)
            #self.classifier_auc_train(probs, self.labels)
            self.embeddings.clear()
            self.labels.clear()

            self.log('accuracy', accuracy, on_epoch=True, on_step=False)
            #self.log('classifier.train.auc', self.classifier_auc_train, on_epoch=True, on_step=False)


    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(), lr=6e-2, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]

2023-06-04 19:45:49.657235: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 19:45:50.508253: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [65]:
model = SimCLRModel(backbone, projection_head)
trainer = pl.Trainer(max_epochs=50, devices=1)
trainer.fit(model, train_loader)

/u/home/deutges/.conda/envs/morph_1/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /u/home/deutges/.conda/envs/morph_1/lib/python3.9/si ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]

  | Name                 | Type                 | Params
------

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [11]:
%load_ext autoreload
%autoreload 2
#Dataset+Dataloader
import sys
sys.path.append('../')
from data_scripts.PI_CAI_Detection import PI_CAI_Detection
#sys.path.insert(1,'/u/home/deutges/adlm_ss2023_mmcl/data_scripts')
#from PI_CAI_Detection import PI_CAI_Detection
from torch.utils.data import DataLoader
labels_path='/data1/practical-sose23/morphometric/picai_labels'
image_path='/data1/practical-sose23/morphometric/data'
modalities=["t2w"]
targetVoxelShape= (1.0, 1.0, 3.0)
targetimageSize = [200, 200, 40]
multiModel=False
useAI_Segmentation=True
biasFeildCorrection=False
toShuffle = False
batchSize = 10

dataset = PI_CAI_Detection(image_path,labels_path,modalities,\
                            multiModel,useAI_Segmentation,biasFeildCorrection,\
                            targetVoxelShape,targetimageSize)
generator1 = torch.Generator().manual_seed(42)
train, val, test = utils.data.random_split(dataset, [0.8, 0.1, 0.1], generator=generator1)

train_loader = DataLoader(train,batch_size=batchSize,shuffle=toShuffle)
val_loader = DataLoader(val,batch_size=batchSize,shuffle=toShuffle)
test_loader = DataLoader(test,batch_size=batchSize,shuffle=toShuffle)
#loader = DataLoader(dataset,batch_size=batchSize,shuffle=toShuffle)


#batch=next(iter(loader))
#print(len(batch))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from pytorch_lightning.loggers import TensorBoardLogger
logger = TensorBoardLogger("tb_logs", name = "test")

In [32]:
model = SimCLRModel(backbone, projection_head)
trainer = pl.Trainer(max_epochs=5, devices=1)
trainer.fit(model, train_loader)

/u/home/deutges/.conda/envs/morph_1/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /u/home/deutges/.conda/envs/morph_1/lib/python3.9/si ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]

  | Name                 | Type                 | Params
------

Training: 0it [00:00, ?it/s]

torch.Size([25, 128])


AttributeError: 'numpy.ndarray' object has no attribute 'is_floating_point'

In [13]:
#import tensorboard
#%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

In [7]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

encoder=model
encoder.projection_head=Identity()
encoder.criterion=Identity()
classifier = nn.Sequential(nn.Linear(512,1),nn.Sigmoid())

class Simple_Classifier(pl.LightningModule):
    def __init__(self, encoder, classifier, loss=nn.BCELoss()):
        super().__init__()
        self.encoder = encoder
        self.classifier = classifier
        self.loss = loss
        #self.accuracy = Accuracy(task="binary")

    def forward(self, x):
        self.encoder.eval()
        with torch.no_grad():
            h = self.encoder(x).flatten(start_dim=1)
        z = self.classifier(h).squeeze().float()
        return z
    
    def training_step(self, batch, batch_idx):
        _, x,y  = batch
        x = x[:,None,:,:,:]
        z = self.forward(x)
        loss = self.loss(z.float(), y.float())
        #self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=False)
        #self.log={"train_loss": loss}
        self.log_dict({"train_loss": loss},on_step=False,on_epoch=True,prog_bar=True)
        return {"loss": loss}
    
    def validation_step(self, batch, batch_idx):
        _, x,y  = batch
        x = x[:,None,:,:,:]
        z = self.forward(x)
        loss = self.loss(z.float(), y.float())
        #self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=False)
        #self.log={"train_loss": loss}
        self.log_dict({"val_loss": loss},on_step=False,on_epoch=True,prog_bar=True)
        return {"loss": loss}

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


# init the autoencoder
classifier_model = Simple_Classifier(encoder,classifier)

NameError: name 'model' is not defined

In [80]:
classifier_model = Simple_Classifier(encoder,classifier)
trainer = pl.Trainer(max_epochs=1, devices=1)
trainer.fit(classifier_model, train_loader,val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [14]:

resnet = resnet10(pretrained=False,spatial_dims=3,n_input_channels=1,num_classes=1)

class Baseline_Classifier(pl.LightningModule):
    def __init__(self, encoder, loss=nn.BCELoss()):
        super().__init__()
        self.encoder = encoder
        self.sigma = nn.Sigmoid()
        self.loss = loss

    def forward(self, x):
        pred = self.encoder(x).squeeze()
        pred = self.sigma(pred).float()
        return pred
    
    def training_step(self, batch, batch_idx):
        x, _,y  = batch
        x = x[:,None,:,:,:]
        z = self.forward(x)
        loss = self.loss(z.float(), y.float())
        self.log_dict({"train_loss": loss},on_step=False,on_epoch=True,prog_bar=True)
        return {"loss": loss}
    
    def validation_step(self, batch, batch_idx):
        _, x,y  = batch
        x = x[:,None,:,:,:]
        z = self.forward(x)
        loss = self.loss(z.float(), y.float())
        self.log_dict({"val_loss": loss},on_step=False,on_epoch=True,prog_bar=True)
        return {"loss": loss}

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-2)
        return optimizer


In [16]:
baseline = Baseline_Classifier(resnet)
trainer = pl.Trainer(max_epochs=80, devices=1)
trainer.fit(baseline, train_loader,val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]

  | Name    | Type    | Params
------------------------------------
0 | encoder | ResNet  | 14.4 M
1 | sigma   | Sigmoid | 0     
2 | loss    | BCELoss | 0     
------------------------------------
14.4 M    Trainable params
0         Non-trainable params
14.4 M    Total params
57.428    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]